# 1. Import Data

In [93]:
from datetime import datetime, timedelta
import re
import pandas as pd
import os

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
base_path = '/content/drive/MyDrive/[MVC]공동연구개발/네이버뉴스수집'

In [102]:
# 파일 수정은 여기서
# article_df = pd.read_excel(os.path.join(base_path, '네이버_산업재계_최신순_뉴스_2024-05-02.xlsx'))
article_df = pd.read_excel('/content/drive/MyDrive/Task/MVC/date/data/조매연_2.xlsx')

In [97]:
# pd.to_datetime(article_df['생성일자'])
# article_df['생성일자'].astype(str)

article_df.head()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Source.Name   48 non-null     object        
 1   기사제목          48 non-null     object        
 2   생성일자          48 non-null     datetime64[ns]
 3   언론사           48 non-null     object        
 4   journalist    3 non-null      object        
 5   url           48 non-null     object        
 6   기사_내용         48 non-null     object        
 7   기사_HTML_body  4 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.1+ KB


# 2. Preprocessing

## 2-0. 날짜 치환 함수 정의

In [101]:
def process_dates_in_article(article_date, text):
    # Helper function to find the next Friday
    def find_friday(date):
        current_weekday = date.weekday()
        days_until_friday = 4 - current_weekday
        return date + timedelta(days=days_until_friday)

    # Helper function to adjust the date for "지난 n일" pattern
    def adjust_date(year, month, day):
        if month == 1:
            return datetime(year - 1, 12, day)
        else:
            return datetime(year, month - 1, day)

    # Define the regular expressions and corresponding lambda functions
    regex_patterns = {
        # DAY
        "이날": ("이[- ]?날",
                        lambda match: article_date.strftime('%Y-%m-%d')),
        "어제": ("어제",
                        lambda match: (article_date - timedelta(days=1)).strftime('%Y-%m-%d')),
        "오늘": ("오늘",
                        lambda match: article_date.strftime('%Y-%m-%d')),
        "지난 n일": ("지난[- ]?(\d+)일",
                        lambda match: (adjust_date(article_date.year, article_date.month, int(match.group(1))) if int(match.group(1)) > article_date.day else article_date.replace(day=int(match.group(1)))).strftime('%Y-%m-%d')),
        "지난달 n일": ("지난[- ]?달[- ]?(\d+)일",
                        lambda match: adjust_date(article_date.year, article_date.month, int(match.group(1))).strftime('%Y-%m-%d')),
        "이달 n일": ("이달[- ]?(\d+)일",
                        lambda match: article_date.replace(day=int(match.group(1))).strftime('%Y-%m-%d')),
        "내달 n일": ("내달[- ]?(\d+)일",
                        lambda match: datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1))).strftime('%Y-%m-%d')),
        "오는 n일": ("오는[- ]?(\d+)일",
                        lambda match: (article_date.replace(day=int(match.group(1))) if int(match.group(1)) > article_date.day else datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1)))).strftime('%Y-%m-%d')),
        "올해 n월 n일": ("올해[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "내년 n월 n일": ("내년[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year + 1, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "작년 n월 n일": ("작년[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "n월 n일": ("(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "n일": ("([1-9]|1[0-9]|2[0-9]|3[01])일",
                        lambda match: (datetime(article_date.year - 1 if article_date.month == 1 and int(match.group(1)) > article_date.day else article_date.year,
                                                12 if article_date.month == 1 and int(match.group(1)) > article_date.day else article_date.month - 1 if int(match.group(1)) > article_date.day else article_date.month,
                                                int(match.group(1)))).strftime('%Y-%m-%d')),

        # WEEK
        "이번주": ("이번[- ]?주",
                        lambda match: find_friday(article_date).strftime('%Y-%m')),
        "다음주": ("다음[- ]?주",
                        lambda match: (find_friday(article_date) + timedelta(days=7)).strftime('%Y-%m')),
        "지난주": ("지난[- ]?주",
                        lambda match: (find_friday(article_date) - timedelta(days=7)).strftime('%Y-%m')),


        # MONTH
        "작년 n월": ("작년[- ]?(\d+)월",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), 1).strftime('%Y-%m')),
        "지난해 n월": ("지난[- ]?해 (\d+)월",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), 1).strftime('%Y-%m')),
        "지난 n월": ("지난[- ]?(\d+)월",
                        lambda match: datetime(article_date.year - 1 if article_date.month < int(match.group(1)) else article_date.year, int(match.group(1)), 1).strftime('%Y-%m')),
        "지난달": ("지난[- ]?달",
                        lambda match: (article_date.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m')),
        "이전달": ("이전[- ]?달",
                        lambda match: (article_date.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m')),
        "이달": ("이달",
                        lambda match: article_date.strftime('%Y-%m')),

        # 분기(Q)
        "YYYY년 N분기": ("(\d{4})년[- ]?(\d{1})분기",
                        lambda match: match.group(1) + '-' + match.group(2) + 'Q'),
        "지난해 N분기": ("지난해[- ]?(\d{1})분기",
                        lambda match: str(article_date.year - 1) + '-' + match.group(1) + 'Q'),
        "올해 N분기": ("올해[- ]?(\d{1})분기",
                        lambda match: str(article_date.year) + '-' + match.group(1) + 'Q'),
        "올 N분기": ("올[- ]?(\d{1})분기",
                        lambda match: str(article_date.year) + '-' + match.group(1) + 'Q'),

        # 상반기(1H)
        "YYYY년 상반기": ("(\d{4})년[- ]?상반기",
                        lambda match: match.group(1) + '-1H'),
        "지난해 상반기": ("지난해[- ]?상반기",
                        lambda match: str(article_date.year - 1) + '-1H'),
        "올해 상반기": ("올해[- ]?상반기",
                        lambda match: str(article_date.year) + '-1H'),
        "올 상반기": ("올[- ]?상반기",
                        lambda match: str(article_date.year) + '-1H'),
        # 하반기(2H)
        "YYYY년 하반기": ("(\d{4})년[- ]?하반기",
                        lambda match: match.group(1) + '-2H'),
        "지난해 하반기": ("지난해[- ]?하반기",
                        lambda match: str(article_date.year - 1) + '-2H'),
        "올해 하반기": ("올해[- ]?하반기",
                        lambda match: str(article_date.year) + '-2H'),
        "올 하반기": ("올[- ]?하반기",
                        lambda match: str(article_date.year) + '-2H'),

        # YEAR
        "올해": ("올해",
                        lambda match: article_date.strftime('%Y년')),
        "작년": ("작년",
                        lambda match: datetime(article_date.year - 1, 1, 1).strftime('%Y년')),
        "지난해": ("지난[- ]?해",
                        lambda match: datetime(article_date.year - 1, 1, 1).strftime('%Y년')),
        "내년": ("내년",
                        lambda match: datetime(article_date.year + 1, 1, 1).strftime('%Y년'))
    }

    # Replace patterns in the text with the calculated date strings
    for key, (pattern_str, replacement_func) in regex_patterns.items():
        pattern = re.compile(pattern_str)
        # For "지난 n일", special handling to adjust the month correctly
        if key == "지난 n일":
            def replacement(match):
                day = int(match.group(1))
                if day > article_date.day:
                    new_date = adjust_date(article_date.year, article_date.month, day)
                    return new_date.strftime('%Y-%m-%d')
                else:
                    return article_date.replace(day=day).strftime('%Y-%m-%d')
            text = pattern.sub(replacement, text)
        else:
            text = pattern.sub(replacement_func, text)

    # Helper function to reformat date strings
    def reformat_dates(text):
        pattern = r'\d{4}-\d{2}(?:-\d{2})?'

        def format_match(match):
            date_str = match.group(0)
            try:
                if '-' in date_str[7:]:  # Looks for the second '-' in 'YYYY-MM-DD'
                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    return date_obj.strftime('%Y년 %m월 %d일')
                else:
                    date_obj = datetime.strptime(date_str, '%Y-%m')
                    return date_obj.strftime('%Y년 %m월')
            except ValueError:
                return date_str

        return re.sub(pattern, format_match, text)

    # Reformat dates in the processed text
    formatted_text = reformat_dates(text)

    return formatted_text

# # Example usage
# article_date = datetime(2023, 5, 6)
# text = "이날 어제 오늘 지난 5일 지난달 3일 지난주 이번주 다음주 올해 5월 1일"
# formatted_text = process_dates_in_article(article_date, text)
# print(formatted_text)


## 2-1. `기사_내용` 날짜 치환

In [99]:
def apply_process_dates(row):
    return process_dates_in_article(row['생성일자'], row['기사_내용'])

In [103]:
article_df['기사_내용_date_replacement'] = article_df.apply(apply_process_dates, axis=1)

## 2-2. `기사제목`, `생성일자`, `언론사`와 concat해서 새로운 열 생성

In [104]:
article_df['GPT_input_text'] = "기사 제목: " + article_df['기사제목'].map(str) + "\n" + "기사 발행일자: " + article_df['생성일자'].map(str) + "\n" + "언론사: " + article_df['언론사'].map(str) + "\n" + "기사 내용: " + article_df['기사_내용'].map(str)

# Export Data

In [105]:
# article_df.to_excel('/content/drive/MyDrive/Task/MVC/date/output/조매연_2_replacement.xlsx')